# 读取DataPattern下的文件

注意：所有JPG图像都会被认为是训练集。

In [ ]:
import os
import pandas as pd

from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd

data_pattern = get_param_in_cwd('data_pattern')
patches = []
for r, ds, fs in os.walk(data_pattern):
    patches.extend([os.path.join(r, p) for p in fs if p.endswith('.jpg')])

patches = pd.DataFrame(patches, columns=['fpath'])
patches['fname'] = patches['fpath'].map(lambda x: os.path.basename(x))

def map2id(x):
    x = os.path.basename(x)
    *fname_items, w, h, ltx, lty, rbx, rby = x.split('_')
    return '_'.join(fname_items)
patches['ID'] = patches['fpath'].map(map2id)
patches

# 获取划分的数据集

In [ ]:
label_f = pd.read_csv(get_param_in_cwd('label_file'))
label_f['ID'] = label_f['ID'].map(lambda x: str(x).replace('.nii.gz', ''))
patches = pd.merge(patches, label_f, on='ID', how='inner')
patches

# 保存List模式数据

In [ ]:
import numpy as np

train_patches = patches[patches['group'] == 'train']
train_patches[['fpath', 'label']].to_csv(os.path.join('split_info', f'train.txt'), index=False, header=False, sep='\t')
test_patches = patches[patches['group'] != 'train']
test_patches[['fpath', 'label']].to_csv(os.path.join('split_info', f'val.txt'), index=False, header=False, sep='\t')